In [1]:
import json
import requests

In [2]:
#https://pypi.org/project/gamma-viewer/
from gamma_viewer import GammaViewer
from IPython.display import display

In [3]:
def submit_to_ars(m):
    submit_url = 'https://ars.transltr.io/ars/api/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'https://arax.ncats.io/?source=ARS&id={message_id}')
    return message_id

In [4]:
def retrieve_ars_results(mid):
    message_url = f'https://ars.transltr.io/ars/api/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    for child in j['children']:
        if child['status'] == 'Done':
            childmessage_id = child['message']
            child_url = f'https://ars.transltr.io/ars/api/messages/{childmessage_id}'
            child_response = requests.get(child_url).json()
            nresults = len(child_response['fields']['data']['message']['results'])
            if nresults > 0:
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
        else:
            nresults = 0
        print( child['status'], child['actor']['agent'],nresults )
    return results

In [5]:
def printjson(j):
    print(json.dumps(j,indent=4))

In [6]:
def name_lookup(text):
    url= f'https://name-resolution-sri.renci.org/lookup?string={text}&offset=0&limit=10'
    response = requests.post(url)
    printjson(response.json())

In [30]:
def one_hop_message(subject_category,object_category,predicate=None,subject_id = None, object_id = None):
    query_graph = {
    "nodes": {
        'a':{
            "category": subject_category
        },
        'b':{
            "category": object_category
        }
    },
    "edges": {
        'ab':{
            "subject": "a",
            "object": "b",
            #"predicate": predicate
        }
    }
    }
    if predicate is not None:
        query_graph['edges']['ab']['predicate'] = predicate
    if subject_id is not None:
        query_graph['nodes']['a']['id'] = subject_id
    if object_id is not None:
        query_graph['nodes']['b']['id'] = object_id
    message = {"message": {"query_graph": query_graph},'knowledge_graph':{"nodes": [], "edges": [],}, 'results':[]}
    return message

In [9]:
name_lookup('glycerol kinase')

{
    "www:bget?hsa:2710": [
        "GKD; glycerol kinase",
        "K00864 glycerol kinase [EC:2.7.1.30] | (RefSeq) GK",
        "GK1"
    ],
    "www:bget?hsa:2712": [
        "GKTA; glycerol kinase 2",
        "K00864 glycerol kinase [EC:2.7.1.30] | (RefSeq) GK2",
        "GKP2"
    ],
    "www:bget?hsa:55750": [
        "MULK; acylglycerol kinase",
        "K09881 acylglycerol kinase [EC:2.7.1.94] | (RefSeq) AGK",
        "CATC5",
        "CTRCT38",
        "MTDPS10"
    ],
    "MONDO:0010613": [
        "glycerol kinase deficiency",
        "glycerol kinase deficiency; GKD",
        "inborn error of glycerol kinase activity",
        "inborn glycerol kinase activity disorder",
        "rare inborn error of glycerol kinase activity",
        "rare inborn error of glycerol kinase activity",
        "GKD",
        "GK deficiency",
        "GK1 deficiency",
        "hyperglycerolemia"
    ],
    "www:bget?ko:K00864": [
        "glycerol kinase [EC:2.7.1.30]",
        "GK",
        "g

In [13]:
m = one_hop_message('biolink:Disease','biolink:PhenotypicFeature','biolink:has_phenotype',subject_id='MONDO:0010613')

In [14]:
printjson(m)

{
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "category": "biolink:Disease",
                    "id": "MONDO:0010613"
                },
                "b": {
                    "category": "biolink:PhenotypicFeature"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b",
                    "predicate": "biolink:has_phenotype"
                }
            }
        }
    },
    "knowledge_graph": {
        "nodes": [],
        "edges": []
    },
    "results": []
}


In [15]:
mid = submit_to_ars(m)

https://arax.ncats.io/?source=ARS&id=3632b833-af19-452e-ad0c-35108fe6ef58


In [16]:
j = retrieve_ars_results(mid)

Done
Done ara-aragorn 79


KeyError: 'results'

In [25]:
view = GammaViewer(props={"data":j['ara-arax']})

In [28]:
j['ara-arax']['message']['results'][1]

{'confidence': 0.8571428571428571,
 'description': 'No description available',
 'edge_bindings': {'N1': [{'id': 'N1_0'}], 'ab': [{'id': 'KG2c:31961106'}]},
 'essence': 'BCKDHB',
 'essence_category': 'biolink:Gene',
 'node_bindings': {'a': [{'id': 'UniProtKB:P21953'}],
  'b': [{'id': 'MONDO:0009563'}]},
 'reasoner_id': 'ARAX',
 'row_data': [0.857, 'BCKDHB', 'biolink:Gene']}

In [29]:
bteview = GammaViewer(props={"data":j['ara-bte']})

In [30]:
display(bteview)

In [17]:
m = one_hop_message('biolink:Disease','biolink:PhenotypicFeature','biolink:has_phenotype',subject_id='OMIM:307030')

In [18]:
printjson(m)

{
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "category": "biolink:Disease",
                    "id": "OMIM:307030"
                },
                "b": {
                    "category": "biolink:PhenotypicFeature"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b",
                    "predicate": "biolink:has_phenotype"
                }
            }
        }
    },
    "knowledge_graph": {
        "nodes": [],
        "edges": []
    },
    "results": []
}


In [19]:
mid = submit_to_ars(m)

https://arax.ncats.io/?source=ARS&id=0d2105d9-d23f-4814-9492-8294c9487a3d


In [20]:
m = one_hop_message('biolink:Disease','biolink:Gene','biolink:caused_by',subject_id='OMIM:307030')

In [21]:
mid = submit_to_ars(m)

https://arax.ncats.io/?source=ARS&id=d40cae6e-bece-4baa-afa7-b39c198bf8f5


In [23]:
printjson(m)

{
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "category": "biolink:Disease",
                    "id": "OMIM:307030"
                },
                "b": {
                    "category": "biolink:Gene"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b",
                    "predicate": "biolink:caused_by"
                }
            }
        }
    },
    "knowledge_graph": {
        "nodes": [],
        "edges": []
    },
    "results": []
}


In [24]:
m = one_hop_message('biolink:Gene','biolink:Disease','biolink:causes',object_id='OMIM:307030')

In [26]:
printjson(m)

{
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "category": "biolink:Gene"
                },
                "b": {
                    "category": "biolink:Disease",
                    "id": "OMIM:307030"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b",
                    "predicate": "biolink:causes"
                }
            }
        }
    },
    "knowledge_graph": {
        "nodes": [],
        "edges": []
    },
    "results": []
}


In [27]:
mid = submit_to_ars(m)

https://arax.ncats.io/?source=ARS&id=4f782ac6-6f82-478a-bfad-2c88adea0b95


In [31]:
m = one_hop_message('biolink:Disease','biolink:Pathway',subject_id='OMIM:307030')

In [32]:
printjson(m)

{
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "category": "biolink:Disease",
                    "id": "OMIM:307030"
                },
                "b": {
                    "category": "biolink:Pathway"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b"
                }
            }
        }
    },
    "knowledge_graph": {
        "nodes": [],
        "edges": []
    },
    "results": []
}


In [33]:
mid = submit_to_ars(m)

https://arax.ncats.io/?source=ARS&id=5b326466-90fe-4811-8127-808dd01872ce


In [35]:
m = one_hop_message('biolink:NamedThing','biolink:Pathway',predicate='biolink:participates_in',object_id='PathWhiz:PW000204')

In [36]:
printjson(m)

{
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "category": "biolink:ChemicalSubstance"
                },
                "b": {
                    "category": "biolink:Pathway",
                    "id": "PathWhiz:PW000204"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b",
                    "predicate": "biolink:participates_in"
                }
            }
        }
    },
    "knowledge_graph": {
        "nodes": [],
        "edges": []
    },
    "results": []
}


In [38]:
mid = submit_to_ars(m)

https://arax.ncats.io/?source=ARS&id=a400f348-7126-42e8-a840-18bb8ac83f7d


In [39]:
m = one_hop_message('biolink:ChemicalSubstance','biolink:Pathway',predicate='biolink:participates_in',object_id='SMPDB:SMP0000187')

In [40]:
mid = submit_to_ars(m)

https://arax.ncats.io/?source=ARS&id=927866d4-73c5-468c-9823-4d9bd03010c9


In [ ]:
mm={
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                },
                "b": {
                    "category": "biolink:Pathway",
                    "id": "PathWhiz:PW000204"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b",
                    "predicate": "biolink:participates_in"
                }
            }
        }
    },
    "knowledge_graph": {
        "nodes": [],
        "edges": []
    },
    "results": []
}

In [41]:
m = one_hop_message('biolink:ChemicalSubstance','biolink:Gene',object_id='UniPRotKB:P32189')

In [42]:
mid = submit_to_ars(m)

https://arax.ncats.io/?source=ARS&id=ba96a323-ecdb-4213-bb0c-4c1c910981d8


In [43]:
printjson(m)

{
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "category": "biolink:ChemicalSubstance"
                },
                "b": {
                    "category": "biolink:Gene",
                    "id": "UniPRotKB:P32189"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b"
                }
            }
        }
    },
    "knowledge_graph": {
        "nodes": [],
        "edges": []
    },
    "results": []
}
